In [4]:
from flask import Flask, render_template, request, jsonify
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import logging

In [1]:


# Setup logging for debugging
logging.basicConfig(level=logging.DEBUG)

# Load the wine dataset
wine_dataset = pd.read_csv('winequality-red.csv')
wine_dataset.dropna(inplace=True)

# Separate the features and target variable
x = wine_dataset.drop('quality', axis=1)
y = wine_dataset['quality']

# Split the dataset into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=2)

# Initialize and train the RandomForest model
model = RandomForestClassifier()
model.fit(x_train, y_train)

app = Flask(__name__)

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict():
    try:
        data = request.json
        app.logger.debug(f"Received data: {data}")
        
        input_data = np.array([[
            data['fixed_acidity'], data['volatile_acidity'], data['citric_acid'], 
            data['residual_sugar'], data['chlorides'], data['free_sulfur_dioxide'], 
            data['total_sulfur_dioxide'], data['density'], data['pH'], 
            data['sulphates'], data['alcohol']
        ]])

        prediction = model.predict(input_data)

        app.logger.debug(f"Prediction: {prediction}")

        if prediction >= 7:
            quality_label = "best"
        elif 4 <= prediction <= 6:
            quality_label = "good"
        else:
            quality_label = "bad"

        return jsonify({'quality': int(prediction[0]), 'label': quality_label})
    except Exception as e:
        app.logger.error(f"Error: {e}")
        return jsonify({'error': str(e)})

if __name__ == '__main__':
    import os
    if os.environ.get("FLASK_ENV") != "production":
        app.run(debug=True)


NameError: name 'logging' is not defined